## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/94-googleq-a-train-glove-sgd-cyclical-triangular2/model.h5'
tokenizer_path = '/kaggle/input/94-googleq-a-train-glove-sgd-cyclical-triangular2/tokenizer.joblib'
category_encoder_path = '/kaggle/input/94-googleq-a-train-glove-sgd-cyclical-triangular2/category_encoder.joblib'
host_encoder_path = '/kaggle/input/94-googleq-a-train-glove-sgd-cyclical-triangular2/host_encoder.joblib'
scaler_path = '/kaggle/input/94-googleq-a-train-glove-sgd-cyclical-triangular2/scaler.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

    
# Text length
test['question_title_len'] = test['question_title'].apply(lambda x : len(x))
test['question_body_len'] = test['question_body'].apply(lambda x : len(x))
test['answer_len'] = test['answer'].apply(lambda x : len(x))

# Word count
test['question_title_word_cnt'] = test['question_title'].apply(lambda x : len(x.split(' ')))
test['question_body_word_cnt'] = test['question_body'].apply(lambda x : len(x.split(' ')))
test['answer_word_cnt'] = test['answer'].apply(lambda x : len(x.split(' ')))

# Unique word count
test['question_title_unique_word_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' '))))
test['question_body_unique_word_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' '))))
test['answer_unique_word_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' '))))

# Misspelings count
misspelling_mapping_set = set(misspelling_mapping.keys())
test['question_title_misspelings_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['question_body_misspelings_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['answer_misspelings_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))

# Meta features
test['question_title_count'] = test['question_title'].apply(lambda x : x.count('?'))
test['question_body_count'] = test['question_body'].apply(lambda x : x.count('?'))
test['title_exp_count'] = test['question_title'].apply(lambda x : x.count('why'))
test['body_exp_count'] = test['question_body'].apply(lambda x : x.count('why'))
test['title_choice_count'] = test['question_title'].apply(lambda x : x.count('or'))
test['body_choice_count'] = test['question_body'].apply(lambda x : x.count('or'))
test['question_aswer_ratio'] = test['question_title_word_cnt'] / test['answer_word_cnt']
test['same_author'] = test['question_user_page'] == test['answer_user_page']

for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 100
ANSWER_MAX_LEN = 100

## Test set

In [6]:
tokenizer = load(tokenizer_path)
ohe_category = load(category_encoder_path)
ohe_host = load(host_encoder_path)
scaler = load(scaler_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']
X_test_category = ohe_category.transform(test[['category']]).toarray()
X_test_host = ohe_host.transform(test[['host']]).toarray()
X_test_dense = test[['question_title_len', 'question_title_word_cnt', 'question_title_unique_word_cnt', 'question_title_misspelings_cnt', 
                     'question_body_len', 'question_body_word_cnt', 'question_body_unique_word_cnt', 'question_body_misspelings_cnt', 
                     'answer_len', 'answer_word_cnt', 'answer_unique_word_cnt', 'answer_misspelings_cnt', 
                     'question_title_count', 'question_body_count', 'title_exp_count', 'body_exp_count', 'title_choice_count', 
                     'body_choice_count', 'body_choice_count', 'question_aswer_ratio', 'same_author']]

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test_dense = scaler.transform(X_test_dense)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq, X_test_category, X_test_host, X_test_dense]

# Model

In [7]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.3)(title)
title = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(title)
title = Dropout(0.5)(title)
title = Dense(512, kernel_initializer='glorot_uniform')(title)
title = BatchNormalization()(title)
title_out = Activation('relu')(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.3)(body)
body = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(body)
body = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(body)
body = Dropout(0.5)(body)
body = Dense(512, kernel_initializer='glorot_uniform')(body)
body = BatchNormalization()(body)
body_out = Activation('relu')(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.3)(answer)
answer = Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='glorot_uniform'))(answer)
answer = Bidirectional(LSTM(128, kernel_initializer='glorot_uniform'))(answer)
answer = Dropout(0.5)(answer)
answer = Dense(512, kernel_initializer='glorot_uniform')(answer)
answer = BatchNormalization()(answer)
answer_out = Activation('relu')(answer)

# Category
input_category = Input(shape=(X_test_category.shape[1],), dtype=tf.float32, name='input_category')
category = Embedding(X_test_category.shape[1], X_test_category.shape[1]//2)(input_category)
category_out = GlobalAveragePooling1D()(category)

# Host
input_host = Input(shape=(X_test_host.shape[1],), dtype=tf.float32, name='input_host')
host = Embedding(X_test_host.shape[1], X_test_host.shape[1]//2)(input_host)
host_out = GlobalAveragePooling1D()(host)

# Dense features
input_dense = Input(shape=(X_test_dense.shape[1],), dtype=tf.float32, name='input_dense')

# Output
output = Concatenate()([title_out, body_out, answer_out, category_out, host_out, input_dense])
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dropout(0.5)(output)
output = Dense(512, kernel_initializer='glorot_uniform')(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform')(output)

model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq, input_category, input_host, input_dense], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_body_seq (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     17402100    input_body_seq[0][0]             
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.895336,0.586851,0.108036,0.653281,0.805287,0.710063,0.623241,0.578562,0.379568,...,0.847250,0.871748,0.594953,0.938844,0.942367,0.774055,0.107307,0.084791,0.633710,0.888923
1,46,0.831528,0.503114,0.003156,0.677863,0.751262,0.885684,0.523826,0.430404,0.111178,...,0.687243,0.919762,0.626076,0.945747,0.963337,0.827140,0.901823,0.153387,0.132856,0.833390
2,70,0.907767,0.711044,0.081444,0.679501,0.912176,0.819008,0.620105,0.614488,0.324454,...,0.862613,0.822694,0.569940,0.898447,0.922838,0.746645,0.038430,0.048912,0.582280,0.868296
3,132,0.843947,0.486833,0.005175,0.727651,0.738102,0.917102,0.528873,0.437396,0.149198,...,0.702559,0.934860,0.656765,0.959598,0.980612,0.873789,0.826706,0.181517,0.492007,0.881683
4,200,0.902810,0.596786,0.039751,0.735823,0.863333,0.836577,0.605499,0.540292,0.343095,...,0.829449,0.931855,0.650434,0.969194,0.971674,0.859078,0.143616,0.089038,0.761751,0.917561


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.867255,0.558815,0.043134,0.697800,0.780890,0.837961,0.568897,0.484158,0.234161,...,0.756480,0.916527,0.647909,0.954032,0.963427,0.843557,0.564270,0.145851,0.487441,0.888111
std,2812.670060,0.043783,0.123354,0.076361,0.080352,0.087612,0.110304,0.039568,0.069976,0.110698,...,0.087476,0.026637,0.031323,0.015605,0.016994,0.033612,0.307039,0.057766,0.241591,0.028915
min,39.000000,0.720180,0.345741,0.000849,0.369220,0.238523,0.204743,0.496516,0.386109,0.058568,...,0.580719,0.816598,0.539771,0.894648,0.874409,0.704022,0.007348,0.017894,0.068589,0.818911
25%,2572.000000,0.833556,0.448171,0.005786,0.663572,0.741834,0.823010,0.536005,0.437092,0.136243,...,0.682122,0.902439,0.629773,0.944923,0.955047,0.825902,0.271546,0.106339,0.275698,0.868282
50%,5093.000000,0.868020,0.559989,0.011155,0.703221,0.781071,0.879886,0.562779,0.457465,0.223995,...,0.762742,0.921412,0.648059,0.953640,0.967753,0.846913,0.691277,0.147504,0.492414,0.890089
75%,7482.000000,0.900319,0.654243,0.036812,0.749203,0.829204,0.899227,0.600397,0.514486,0.326664,...,0.838158,0.933882,0.668555,0.964580,0.974921,0.865332,0.823176,0.184136,0.693686,0.908864
max,9640.000000,0.962525,0.832620,0.522838,0.891839,0.966868,0.947541,0.670979,0.702255,0.481280,...,0.910075,0.970383,0.714207,0.989874,0.991783,0.918187,0.932020,0.339588,0.922946,0.952245
